In [1]:
from ase.io import read

In [2]:
atoms = read("ABABEM.cif")
print(atoms)

Atoms(symbols='C24H32N8O8S4', pbc=True, cell=[[7.84, 0.0, 0.0], [0.10592431760794835, 10.463463864272638, 0.0], [1.8943683932444588, 4.616465726146425, 9.254394015276839]], spacegroup_kinds=...)


/Users/jou/anaconda3/envs/CartNet_streamlit/lib/python3.9/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(2, setting=1). This may result in wrong setting!
  warnings.warn(


In [3]:
atoms

Atoms(symbols='C24H32N8O8S4', pbc=True, cell=[[7.84, 0.0, 0.0], [0.10592431760794835, 10.463463864272638, 0.0], [1.8943683932444588, 4.616465726146425, 9.254394015276839]], spacegroup_kinds=...)

In [4]:
atoms.get_temperature()

0.0

In [5]:
len(atoms.positions)


76

In [6]:
atoms.positions

array([[ 7.24196156,  8.82893871,  0.37813454],
       [ 2.59833115,  6.25099088,  8.87625948],
       [ 5.74520279,  7.04629193,  1.65746197],
       [ 4.09508992,  8.03363766,  7.59693205],
       [ 3.88893202, 13.65415447,  9.05172279],
       [ 5.95136069,  1.42577512,  0.20267123],
       [ 7.95613575,  6.58139981,  1.39463718],
       [ 1.88415697,  8.49852978,  7.85975684],
       [ 1.30660262,  7.2300632 ,  1.00225087],
       [ 8.53369009,  7.84986639,  8.25214314],
       [ 2.43029479,  6.34069224,  0.71073746],
       [ 7.40999792,  8.73923735,  8.54365655],
       [ 3.23001749,  6.86481536,  0.54693469],
       [ 6.61027522,  8.21511423,  8.70745933],
       [ 4.13385219, 10.43771939,  9.16925359],
       [ 5.70644052,  4.6422102 ,  0.08514042],
       [ 2.65875695,  5.41525897,  1.88234374],
       [ 7.18153576,  9.66467062,  7.37205027],
       [ 2.96160643,  5.92841868,  2.64675669],
       [ 6.87868628,  9.15151091,  6.60763733],
       [ 3.34980144,  4.77258123,  1.657

In [7]:
atoms.get_atomic_numbers()


array([16, 16,  8,  8,  8,  8,  7,  7,  7,  7,  6,  6,  1,  1,  1,  1,  6,
        6,  1,  1,  1,  1,  6,  6,  1,  1,  1,  1,  6,  6,  1,  1,  1,  1,
        6,  6,  6,  6, 16, 16,  8,  8,  8,  8,  7,  7,  7,  7,  6,  6,  1,
        1,  1,  1,  6,  6,  1,  1,  1,  1,  6,  6,  1,  1,  1,  1,  6,  6,
        1,  1,  1,  1,  6,  6,  6,  6])

In [8]:
from CifFile import ReadCif
cif = ReadCif("ABABEM.cif")
cif_data = cif.first_block()

In [9]:
if "_diffrn_ambient_temperature" in cif_data.keys():
    temperature = float(cif_data["_diffrn_ambient_temperature"])
else:
    temperature = None
temperature


293.0

In [11]:
atoms

Atoms(symbols='C24H32N8O8S4', pbc=True, cell=[[7.84, 0.0, 0.0], [0.10592431760794835, 10.463463864272638, 0.0], [1.8943683932444588, 4.616465726146425, 9.254394015276839]], spacegroup_kinds=...)

In [29]:
atoms[0].number

16

In [12]:
atoms.cell.array

array([[ 7.84      ,  0.        ,  0.        ],
       [ 0.10592432, 10.46346386,  0.        ],
       [ 1.89436839,  4.61646573,  9.25439402]])

In [19]:
from torch_geometric.data import Data, Batch
import torch
import numpy as np
from utils import radius_graph_pbc

mean_temp = torch.tensor(192.1785) #training temp mean
std_temp = torch.tensor(81.2135) #training temp std

data = Data()
data.x = torch.tensor(atoms.get_atomic_numbers(), dtype=torch.float32)
data.pos = torch.tensor(atoms.positions, dtype=torch.float32)
data.temperature_og = torch.tensor([temperature], dtype=torch.float32)
data.temperature = (data.temperature_og - mean_temp) / std_temp
data.cell = torch.tensor(atoms.cell.array, dtype=torch.float32).unsqueeze(0)

data.pbc = torch.tensor([True, True, True])
data.natoms = len(atoms)
batch = Batch.from_data_list([data])

edge_index, _, _, edge_attr = radius_graph_pbc(batch, 5.0, 64)
data.cart_dist = torch.norm(edge_attr, dim=-1)
data.cart_dir = torch.nn.functional.normalize(edge_attr, dim=-1)
data.edge_index = edge_index
data.non_H_mask = data.x != 1
delattr(data, "pbc")
delattr(data, "natoms")
data


Data(x=[76], pos=[76, 3], temperature_og=[1], temperature=[1], cell=[1, 3, 3], cart_dist=[3886], cart_dir=[3886, 3], edge_index=[2, 3886], non_H_mask=[76])

In [20]:
batch = Batch.from_data_list([data])
batch

DataBatch(x=[76], pos=[76, 3], temperature_og=[1], temperature=[1], cell=[1, 3, 3], cart_dist=[3886], cart_dir=[3886, 3], edge_index=[2, 3886], non_H_mask=[76], batch=[76], ptr=[2])

In [25]:
atoms.set_array("adp", np.ones((len(atoms),3,3)) * 0.01)

ValueError: Array "adp" has wrong shape (76, 3, 3) != (76,).

In [23]:
atoms.get_array("adp")

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

In [49]:
import numpy as np

In [51]:
np.__version__

'2.0.2'

In [52]:
!pip install "numpy<2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 4.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
